Day 9 Challenge: We get a heightmap like below, each number corresponds to the height of a particular location, 9 is the highest, 0 is the lowest.

Goal: Find the low points - locations that are lower than any of its adjacent locations (up, down, left, right). 

The risk level of a low point is "1 plus its height".

Find the sum of the risk levels of all the low points. 

<img width=100; src="heightmap_example.png">

## Part 1

In [2]:
import numpy as np

In [21]:
def get_neighbors(row_index, column_index, floorboard):
    neighbors = []
    # left neighbor
    if column_index - 1 >= 0:
        neighbors.append(floorboard[row_index, column_index-1])
    # top neighbor
    if row_index - 1 >= 0:
        neighbors.append(floorboard[row_index-1, column_index])
    # right neighbor
    if column_index + 1 < floorboard.shape[1]:
        neighbors.append(floorboard[row_index, column_index+1])
    # below neighbor
    if row_index + 1 < floorboard.shape[0]:
        neighbors.append(floorboard[row_index+1, column_index])
    return neighbors

def check_if_low_point(row_index, column_index, floorboard):
    neighbors = get_neighbors(row_index, column_index, floorboard)
    return np.all((np.ones((1,len(neighbors)))*floorboard[row_index, column_index] - neighbors) < 0)

def part1(file_name):
    with open(file_name, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    lava_floor = np.zeros((len(lines), len(lines[0])), dtype=int)
    for i, line in enumerate(lines):
        for j, char_entry in enumerate(line):
            lava_floor[i, j] = int(char_entry)
    
    display(lava_floor)

    risk_sum = 0
    for i in range(lava_floor.shape[0]):
        for j in range(lava_floor.shape[1]):
            if check_if_low_point(i, j, lava_floor):
                risk_sum += 1 + lava_floor[i, j]
    
    print('risk sum', risk_sum)
            

In [22]:
part1('example.txt')

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

risk sum 15


In [23]:
part1('input.txt')

array([[8, 6, 7, ..., 7, 8, 9],
       [6, 4, 5, ..., 8, 9, 2],
       [5, 3, 5, ..., 9, 9, 0],
       ...,
       [9, 9, 8, ..., 6, 7, 9],
       [9, 8, 9, ..., 8, 8, 9],
       [8, 7, 6, ..., 9, 9, 9]])

risk sum 564


## Part 2

This basin has size 9

<img width=100; src="basin_example.png">

Goal: Find the 3 largest basins and multiply their sizes together. Each low point has a basin.

In [48]:
def get_neighbors(row_index, column_index, floorboard):
    neighbors = []
    # left neighbor
    if column_index - 1 >= 0:
        neighbors.append(floorboard[row_index, column_index-1])
    # top neighbor
    if row_index - 1 >= 0:
        neighbors.append(floorboard[row_index-1, column_index])
    # right neighbor
    if column_index + 1 < floorboard.shape[1]:
        neighbors.append(floorboard[row_index, column_index+1])
    # below neighbor
    if row_index + 1 < floorboard.shape[0]:
        neighbors.append(floorboard[row_index+1, column_index])
    return neighbors

def get_unmarked_basin_neighbors(i, j, floorboard, basinboard):
    neighbors = []
    # left neighbor
    if j - 1 >= 0:
        if floorboard[i, j-1] < 9 and basinboard[i, j-1] == -1:
            neighbors.append((i, j-1))
    # top neighbor
    if i - 1 >= 0:
        if floorboard[i-1, j] < 9 and basinboard[i-1, j] == -1:
            neighbors.append((i-1, j))
    # right neighbor
    if j + 1 < floorboard.shape[1]:
        if floorboard[i, j+1] < 9 and basinboard[i, j+1] == -1:
            neighbors.append((i, j+1))
    # below neighbor
    if i + 1 < floorboard.shape[0]:
        if floorboard[i+1, j] < 9 and basinboard[i+1, j] == -1:
            neighbors.append((i+1, j))
    return neighbors

def check_if_low_point(row_index, column_index, floorboard):
    neighbors = get_neighbors(row_index, column_index, floorboard)
    return np.all((np.ones((1,len(neighbors)))*floorboard[row_index, column_index] - neighbors) < 0)

def mark_basin(row_index, column_index, floorboard, basinboard, basin_nr):
    basinboard[row_index, column_index] = basin_nr
    unmarked_neighbors = get_unmarked_basin_neighbors(row_index, column_index, floorboard, basinboard)
    for neighbor in unmarked_neighbors:
        if basinboard[neighbor] == -1:
            mark_basin(neighbor[0], neighbor[1], floorboard, basinboard, basin_nr)
    return basinboard

def part2(file_name):
    with open(file_name, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    lava_floor = np.zeros((len(lines), len(lines[0])), dtype=int)
    for i, line in enumerate(lines):
        for j, char_entry in enumerate(line):
            lava_floor[i, j] = int(char_entry)
    
    display(lava_floor)

    basin_map = np.ones((lava_floor.shape)) * -1

    for i in range(lava_floor.shape[0]):
        for j in range(lava_floor.shape[1]):
            if check_if_low_point(i, j, lava_floor):
                # find the basin
                basin_nr = np.max(basin_map) + 1
                basin_map = mark_basin(i, j, lava_floor, basin_map, basin_nr)
    
    display(basin_map)

    _, counts = np.unique(basin_map, return_counts=True)
    counts = list(counts)[1:]
    counts.sort(reverse=True)
    print(counts[0]*counts[1]*counts[2])

In [49]:
part2('example.txt')

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

array([[ 0.,  0., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.],
       [ 0., -1.,  2.,  2.,  2., -1.,  1., -1.,  1.,  1.],
       [-1.,  2.,  2.,  2.,  2.,  2., -1.,  3., -1.,  1.],
       [ 2.,  2.,  2.,  2.,  2., -1.,  3.,  3.,  3., -1.],
       [-1.,  2., -1., -1., -1.,  3.,  3.,  3.,  3.,  3.]])

1134


In [50]:
part2('input.txt')

array([[8, 6, 7, ..., 7, 8, 9],
       [6, 4, 5, ..., 8, 9, 2],
       [5, 3, 5, ..., 9, 9, 0],
       ...,
       [9, 9, 8, ..., 6, 7, 9],
       [9, 8, 9, ..., 8, 8, 9],
       [8, 7, 6, ..., 9, 9, 9]])

array([[ 13.,  13.,  13., ...,   4.,   4.,  -1.],
       [ 13.,  13.,  13., ...,   4.,  -1.,  10.],
       [ 13.,  13.,  13., ...,  -1.,  -1.,  10.],
       ...,
       [ -1.,  -1., 219., ..., 228., 228.,  -1.],
       [ -1., 235.,  -1., ..., 228., 228.,  -1.],
       [235., 235., 235., ...,  -1.,  -1.,  -1.]])

1038240
